In [1152]:
file = open('input', 'r')
file_str = file.read()
lines = file_str.splitlines()

## Part 1

In [1174]:
from math import prod

def h2b(s):
    h2b = {'0': '0000','1': '0001','2': '0010','3': '0011',
           '4': '0100','5': '0101','6': '0110','7': '0111',
           '8': '1000','9': '1001','A': '1010','B': '1011',
           'C': '1100','D': '1101','E': '1110','F': '1111'}
    s = ''.join(h2b[c] for c in s)
    return s

def b2d(s):
    s = s[::-1] # reverse string
    decimal = 0
    for i,c in enumerate(s):
        decimal += int(c) * 2 ** i
    return decimal

class Packet:
    def __init__(self, b):
        self.b = b
        self.version = b2d(self.b[:3])
        self.type = b2d(self.b[3:6])
        if self.type == 4:
            self.subpackets = self.parse_literal()
        else:
            self.subpackets = self.parse_operator()

    def parse_literal(self):
        value_str = ''
        length = 6
        parsing = True
        while parsing:
            group = self.b[length:length+5]
            value_str += group[1:]
            length += 5
            if group[0] == '0':
                parsing = False
        self.length = length
        value = b2d(value_str)
        return value
        
    def parse_operator(self):
        len_id = self.b[6]
        subpackets = []
        if len_id == '0':
            self.length = 22 + b2d(self.b[7:22])
            to_read = self.b[22:self.length]

            while to_read:
                subpackets.append(Packet(to_read))
                to_read = to_read[subpackets[-1].length:]
        else:
            num_subpackets = b2d(self.b[7:18])
            to_read = self.b[18:]
            for i in range(num_subpackets):
                subpackets.append(Packet(to_read))
                to_read = to_read[subpackets[-1].length:]
            self.length = 18 + sum(p.length for p in subpackets)
        return subpackets

    def version_sum(self):
        if self.type == 4:
            return self.version
        else:
            return self.version + sum(p.version_sum() for p in self.subpackets)

    def value(self):
        if self.type == 0:
            return sum(p.value() for p in self.subpackets)
        elif self.type == 1:
            return prod(p.value() for p in self.subpackets)
        elif self.type == 2:
            return min(p.value() for p in self.subpackets)
        elif self.type == 3:
            return max(p.value() for p in self.subpackets)
        elif self.type == 4:
            return self.subpackets
        elif self.type == 5:
            return int(self.subpackets[0].value() > self.subpackets[1].value())
        elif self.type == 6:
            return int(self.subpackets[0].value() < self.subpackets[1].value())
        elif self.type == 7:
            return int(self.subpackets[0].value() == self.subpackets[1].value())

In [1176]:
s = h2b(file_str.strip())
p = Packet(s)
print(p.version_sum())

913


## Part 2

In [1177]:
print(p.value())

1510977819698
